In [12]:
import os
import sys
module_path = os.path.abspath(os.path.join("../.."))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from jupyter_notebook import load_parameters 

In [14]:
import pandas as pd

pars = load_parameters()

input_file = pars.get('input')
output_file = pars.get('output')

input_col = pars.get('input_col')
output_col = pars.get('output_col')

In [23]:
dataset = pd.read_csv(input_file, encoding='ISO-8859-1', error_bad_lines=False, sep=";")

#### Process Terminal-Like Code
The goal of this function is to remove tags related ot notebook or terminal-like codes

Example:
```python
In [11]: df
df.append()
Out[11]: 
A B C D 
2000-01-03 -0.59885 -0.18141 -0.68828 -0.77572
2000-01-04 0.83935 0.15993 0.95911 -1.12959

In [12]: df[(df.values > 1.5).any(1)]
Out[12]: 
A B C D 
2000-01-05 2.8021 -0.1086 -1.62114 -0.2017
2000-01-06 0.7167 -0.2671 1.36029 1.7425
```

Should extract only:
```python
df
df.append()
df[(df.values > 1.5).any(1)]
```

In [24]:
import re

def preprocessTerminalLikeCode(code):
    terminalRegex='In \[.+?\]:(.*?)Out\[.+?\]'
    sequence = re.compile(terminalRegex, re.DOTALL)
    terminalCode = sequence.findall(code)
    return [i.lstrip() for i in terminalCode]

import ast

def preprocessCode(code):
    # First Method for pre-processing
    terminalLikeCode = preprocessTerminalLikeCode(code)
    if terminalLikeCode:
        code = '\n'.join(terminalLikeCode)
    return code
    
def preprocessPost(post, col):
    strcodes = post[col]
    # Get the list of codes
    codes = strcodes.split('\n')
    processed = []
    for code in codes:
        processed.append(preprocessCode(code))
    post[output_col] = processed
    return post

processed_dataset = dataset.apply(preprocessPost, axis=1, args=(input_col,))

In [26]:
# processed_dataset.head()

,Unnamed: 0,Unnamed: 0.1,AcceptedAnswerId,AnswerCount,Body,ClosedDate,CommentCount,CommunityOwnedDate,CreationDate,FavoriteCount,...,OwnerUserId,ParentId,PostTypeId,Score,Tags,Title,ViewCount,CodeList,Code,ProcessedCode
0,27,27,7779260.0,2.0,<p>When I try to merge two dataframes by rows ...,NaN,7,NaN,2011-10-15T08:21:17.460,4.0,...,601314.0,NaN,1,25,<python><pandas>,append two data frame with pandas,25479.0,"['bigdata = data1.append(data2)\r\r\n', 'Excep...",['bigdata = data1.append(data2)\r\r\nException...,[bigdata = data1.append(data2)\r\r\nException:...
1,31,31,11617194.0,7.0,<p>I want to perform my own complex operations...,NaN,3,NaN,2011-10-20T14:46:14.633,108.0,...,1005409.0,NaN,1,187,<python><performance><for-loop><pandas>,What is the most efficient way to loop through...,195364.0,"['Date,Open,High,Low,Close,Volume,Adj Close\r\...","[""Date,Open,High,Low,Close,Volume,Adj Close\r\...","[Date,Open,High,Low,Close,Volume,Adj Close\r\r..."
2,49,49,8916746.0,2.0,<p>I have a dataframe <code>df</code> in panda...,NaN,0,NaN,2012-01-18T19:41:27.017,12.0,...,248237.0,NaN,1,26,<python><csv><numpy><tab-delimited><pandas>,selecting across multiple columns with python ...,19805.0,"['df_greater_than10 = df[df[""colA""] > 10]\r\r\n']","['df_greater_than10 = df[df[""colA""] > 10]\r\r\n']","[df_greater_than10 = df[df[""colA""] > 10]\r\r\n]"
3,54,54,8997908.0,3.0,"<p>I recently came across the <a href=""http://...",NaN,16,NaN,2012-01-24T17:59:53.850,58.0,...,345660.0,NaN,1,134,<python><r><join><data.table><pandas>,Why are pandas merges in python faster than da...,16173.0,[],[''],[]
4,73,73,9557319.0,8.0,<p>I used Enthought's python distribution as a...,2013-12-04T20:55:59.357,5,NaN,2012-03-04T14:25:36.287,6.0,...,402468.0,NaN,1,19,<python><numpy><scipy><enthought><pandas>,Open source Enthought Python alternative,6585.0,[],[''],[]


In [27]:
processed_dataset.to_csv(output_file, encoding='ISO-8859-1', sep=';', doublequote=True, index=False)